In [1]:
# ##The dataset is loaded to your GDrive so need to be mounted
# from google.colab import drive
# import os
# drive.mount('/content/drive')
# os.chdir('/content/drive/My Drive/Colab Notebooks/Satellite Project/')

In [2]:
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from utils import *

from models.basic_unet import UNet
from models.siam_nested_unet import Siam_NestedUNet_Conc

from datasets.dataset import ChangeDetectionDataset
from datasets.diff_dataset import ChangeDetectionDiffDataset

## Read Data


In [3]:
model_name = 'siamese.pt'
testset_path = './testset/'
predicted_images_dir = './testset/Predictions'

In [4]:
after_images = read_data(testset_path, 'after')
before_images = read_data(testset_path,'before')
labels = read_data('./trainval/','label',4800)

Time taken to read  after  images: 0.03 seconds
Time taken to read  before  images: 0.01 seconds
Time taken to read  label  images: 1.10 seconds


In [5]:
# diff_images = preprocess(before_images, after_images, save_dir=testset_path)

In [6]:
print(len(after_images))

1


In [7]:
batch_size = 8

In [8]:
# Define transformations
transform_rgb = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Assuming you have instantiated your dataset like this
# test_dataset = ChangeDetectionDiffDataset(diff_images, diff_images, transform_rgb)
test_dataset = ChangeDetectionDataset(before_images, after_images, labels, transform_rgb)


# Define data loaders for train and validation sets
test_loader = DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle = True)

In [9]:
# Create an instance of the model
threshold = 0.3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# must be changed for each model
# model = UNet(n_channels = 3, n_classes = 1).to(device)
model = Siam_NestedUNet_Conc(in_ch=3, out_ch=1).to(device)
criterion = nn.BCEWithLogitsLoss()

print(device)

cuda


In [10]:
# Load the latest saved model
model = load_model(model, save_dir="saved_models", model_name=model_name, device=device)

In [11]:
predicted_masks_list = predict(model, test_loader, device, threshold)
# predicted_masks_list = predict_diff(model, test_loader, device, threshold)

100%|██████████| 1/1 [00:08<00:00,  8.57s/it]


In [16]:
true_masks_list = [label for _, _, label in test_dataset]

In [17]:
print(len(predicted_masks_list))
print(len(true_masks_list))

1
1


In [12]:
jaccard, overall_accuracy = evaluate(predicted_masks_list, true_masks_list)
print("Validation")
print(f"Mean Jaccard Index :{jaccard*100:.5f}%")
print(f"Overall Accuracy :{overall_accuracy*100:.5f}%")

ValueError: Classification metrics can't handle a mix of unknown and multilabel-indicator targets

In [ ]:
save_predictions(predicted_masks_list)

Saving predictions
Saving Completed
